In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from jurity.recommenders import BinaryRecoMetrics, RankingRecoMetrics
from jurity.classification import BinaryClassificationMetrics



# Simple Bandit

In [3]:
mac = pickle.load(open(os.path.join('output', 'mac_expectations.pkl'), 'rb'))
win = pickle.load(open(os.path.join('output', 'win_expectations.pkl'), 'rb'))
sgm = pickle.load(open(os.path.join('output', 'sgm_expectations.pkl'), 'rb'))
rh = pickle.load(open(os.path.join('output', 'rh_expectations.pkl'), 'rb'))
dar = pickle.load(open(os.path.join('output', 'dar_expectations.pkl'), 'rb'))

In [4]:
responses = pd.read_csv('simulated_data.csv')
test = responses[responses['set']=='test']
test.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,arm,reward,set
280,-0.437539,1.579266,1.855246,6.049628,0.178026,-1.985470,0.005097,0.571541,1.184153,0.250244,...,-0.815639,-2.154333,-0.302746,-2.941561,-1.879810,1.794686,1.415656,0,1,test
281,-1.483903,2.533863,1.334310,-2.786843,-3.153749,-10.011978,-1.486536,2.942048,1.658577,0.282615,...,-7.705325,-2.362107,-0.077421,1.519168,-0.288540,0.164909,-3.257780,2,0,test
282,-1.393583,5.561957,-0.440106,-0.586712,3.047809,-1.553686,1.784540,-0.724047,3.457133,-0.520384,...,-0.580570,0.049443,0.357533,-1.653148,-0.676257,3.890172,-1.096128,3,0,test
283,-4.979037,1.186080,-0.970855,-1.207987,-0.060639,0.088849,-1.570659,1.031042,-0.470511,-1.387079,...,0.173680,-0.856460,-1.225014,0.115370,0.317427,0.476862,3.814958,1,0,test
284,-0.785594,0.057483,-1.800295,-4.470890,-0.046001,-3.470062,-3.025259,0.005172,-2.017571,-1.083593,...,4.259867,2.013612,-0.559288,-3.612187,-0.884423,-0.787468,-1.185029,2,0,test


In [5]:
test = test[['arm', 'reward']]

In [6]:
test['user'] = [i for i in range(test.shape[0])]

In [7]:
test.reset_index(inplace=True, drop=True)

In [8]:
mac_df = pd.DataFrame(mac)
win_df = pd.DataFrame(win)
sgm_df = pd.DataFrame(sgm)
rh_df = pd.DataFrame(rh)
dar_df = pd.DataFrame(dar)

mac_df = mac_df.merge(test, how='left', left_index=True, right_index=True)
win_df = win_df.merge(test, how='left', left_index=True, right_index=True)
sgm_df = sgm_df.merge(test, how='left', left_index=True, right_index=True)
rh_df = rh_df.merge(test, how='left', left_index=True, right_index=True)
dar_df = dar_df.merge(test, how='left', left_index=True, right_index=True)

mac_df.head()

,0,1,2,3,arm,reward,user
0,0.462712,-3.843000,1.176226,-4.607192,0,1,0
1,-1.159344,1.850469,-0.257948,-2.803339,2,0,1
2,-0.366885,-7.232940,1.380116,-0.026947,3,0,2
3,0.276632,-0.183199,1.484589,-0.064278,1,0,3
4,0.858629,-2.174916,-0.509416,-0.851353,2,0,4


In [9]:
mac_df = mac_df.melt(id_vars=['user', 'arm', 'reward'], var_name='item_id', value_name='raw_score')
win_df = win_df.melt(id_vars=['user', 'arm', 'reward'], var_name='item_id', value_name='raw_score')
sgm_df = sgm_df.melt(id_vars=['user', 'arm', 'reward'], var_name='item_id', value_name='raw_score')
rh_df = rh_df.melt(id_vars=['user', 'arm', 'reward'], var_name='item_id', value_name='raw_score')
dar_df = dar_df.melt(id_vars=['user', 'arm', 'reward'], var_name='item_id', value_name='raw_score')

In [10]:
mac_df.head()

,user,arm,reward,item_id,raw_score
0,0,0,1,0,0.462712
1,1,2,0,0,-1.159344
2,2,3,0,0,-0.366885
3,3,1,0,0,0.276632
4,4,2,0,0,0.858629


In [14]:
mac_df['probability'] = mac_df.apply(lambda x: 1/(1 + np.exp(-x['raw_score'])), axis=1)

In [15]:
mac_df.head()

,user,arm,reward,item_id,raw_score,probability
0,0,0,1,0,0.462712,0.613657
1,1,2,0,0,-1.159344,0.238787
2,2,3,0,0,-0.366885,0.409294
3,3,1,0,0,0.276632,0.568720
4,4,2,0,0,0.858629,0.702374


In [16]:
mac_df['probability'].describe()

count    4.800000e+02
mean     5.235783e-01
std      3.327996e-01
min      6.527233e-07
25%      2.311172e-01
50%      5.490153e-01
75%      8.237958e-01
max      9.999999e-01
Name: probability, dtype: float64

In [17]:
win_df['probability'] = win_df.apply(lambda x: 1/(1 + np.exp(-x['raw_score'])), axis=1)
sgm_df['probability'] = sgm_df.apply(lambda x: 1/(1 + np.exp(-x['raw_score'])), axis=1)
rh_df['probability'] = rh_df.apply(lambda x: 1/(1 + np.exp(-x['raw_score'])), axis=1)
dar_df['probability'] = dar_df.apply(lambda x: 1/(1 + np.exp(-x['raw_score'])), axis=1)

In [18]:
mac_df['pred'] = mac_df['probability'] >= 0.5
win_df['pred'] = win_df['probability'] >= 0.5
sgm_df['pred'] = sgm_df['probability'] >= 0.5
rh_df['pred'] = rh_df['probability'] >= 0.5
dar_df['pred'] = dar_df['probability'] >= 0.5

In [19]:
mac_df.head()

,user,arm,reward,item_id,raw_score,probability,pred
0,0,0,1,0,0.462712,0.613657,True
1,1,2,0,0,-1.159344,0.238787,False
2,2,3,0,0,-0.366885,0.409294,False
3,3,1,0,0,0.276632,0.568720,True
4,4,2,0,0,0.858629,0.702374,True


# Table 6, Clicks Column

In [20]:
mac_df['pred'].sum()

259

In [21]:
win_df['pred'].sum()

261

In [22]:
sgm_df['pred'].sum()

259

In [23]:
rh_df['pred'].sum()

259

In [24]:
dar_df['pred'].sum()

256

In [25]:
mac_df.shape[0]

480

# Table 6, Accuracy, Precision, Recall, and AUC Columns

In [26]:
scores = [('Mac', mac_df), ('Windows', win_df), ('SageMaker', sgm_df), ('Red Hat', rh_df), ('Darwin', dar_df)]
for name, df in scores:

    temp_idx = df['arm'] == df['item_id']
    temp = df[temp_idx]
    acc = BinaryClassificationMetrics.Accuracy().get_score(temp['pred'].tolist(), temp['reward'].tolist())
    f1 = BinaryClassificationMetrics.F1().get_score(temp['pred'].tolist(), temp['reward'].tolist())
    precision = BinaryClassificationMetrics.Precision().get_score(temp['pred'].tolist(), temp['reward'].tolist())
    recall = BinaryClassificationMetrics.Recall().get_score(temp['pred'].tolist(), temp['reward'].tolist())
    auc = BinaryClassificationMetrics.AUC().get_score(temp['pred'].tolist(), temp['reward'].tolist())
    print(name, ':', acc, f1, precision, recall, auc)

Mac : 0.65 0.6818181818181819 0.7758620689655172 0.6081081081081081 0.6627497062279673
Windows : 0.65 0.6818181818181819 0.7758620689655172 0.6081081081081081 0.6627497062279673
SageMaker : 0.65 0.6818181818181819 0.7758620689655172 0.6081081081081081 0.6627497062279673
Red Hat : 0.65 0.6818181818181819 0.7758620689655172 0.6081081081081081 0.6627497062279673
Darwin : 0.65 0.671875 0.7413793103448276 0.6142857142857143 0.6571428571428571


In [31]:
df_full = None

for name, df in scores:
    df_temp = df[['user', 'item_id', 'raw_score', 'probability', 'pred']]
    df_temp.columns = [name + c if c not in ['user', 'item_id'] else c for c in df_temp.columns]
    
    if df_full is None:
        df_full = df_temp
    else:
        df_full = df_full.merge(df_temp, how='left', on=['user', 'item_id'])

In [32]:
df_full.head()

,user,item_id,Macraw_score,Macprobability,Macpred,Windowsraw_score,Windowsprobability,Windowspred,SageMakerraw_score,SageMakerprobability,SageMakerpred,Red Hatraw_score,Red Hatprobability,Red Hatpred,Darwinraw_score,Darwinprobability,Darwinpred
0,0,0,0.462712,0.613657,True,0.462712,0.613657,True,0.462712,0.613657,True,0.462712,0.613657,True,0.189958,0.547347,True
1,1,0,-1.159344,0.238787,False,-0.829212,0.303812,False,-1.160620,0.238555,False,-1.159344,0.238787,False,-1.097789,0.250154,False
2,2,0,-0.366885,0.409294,False,0.257145,0.563934,True,-0.369199,0.408735,False,-0.366885,0.409294,False,-0.441364,0.391416,False
3,3,0,0.276632,0.568720,True,-0.046596,0.488353,False,0.277848,0.569019,True,0.276632,0.568720,True,0.209011,0.552063,True
4,4,0,0.858629,0.702374,True,0.748110,0.678767,True,0.858976,0.702447,True,0.858629,0.702374,True,0.331765,0.582189,True


# Table 5, Score Comparisons

In [33]:
results = []
env = ['Mac', 'Windows', 'SageMaker', 'Red Hat', 'Darwin']

for i, name_i in enumerate(env):
    a = name_i + 'raw_score'
    for j, name_j in enumerate(env):
        if j > i:
            b = name_j + 'raw_score'
            similarity = np.isclose(df_full[a], df_full[b])
            results.append((name_i, name_j, similarity))

In [36]:
for a, b, c in results:
    print(a, b, c.shape, str(c.sum()))

Mac Windows (480,) 129
Mac SageMaker (480,) 130
Mac Red Hat (480,) 480
Mac Darwin (480,) 64
Windows SageMaker (480,) 214
Windows Red Hat (480,) 129
Windows Darwin (480,) 64
SageMaker Red Hat (480,) 130
SageMaker Darwin (480,) 64
Red Hat Darwin (480,) 64


# Table 5, Probability Comparisons

In [37]:
results2 = []
env = ['Mac', 'Windows', 'SageMaker', 'Red Hat', 'Darwin']

for i, name_i in enumerate(env):
    a = name_i + 'probability'
    for j, name_j in enumerate(env):
        if j > i:
            b = name_j + 'probability'
            similarity = np.isclose(df_full[a], df_full[b])
            results2.append((name_i, name_j, similarity))
            
for a, b, c in results2:
    print(a, b, c.shape, str(c.sum()))

Mac Windows (480,) 162
Mac SageMaker (480,) 168
Mac Red Hat (480,) 480
Mac Darwin (480,) 69
Windows SageMaker (480,) 238
Windows Red Hat (480,) 162
Windows Darwin (480,) 69
SageMaker Red Hat (480,) 168
SageMaker Darwin (480,) 69
Red Hat Darwin (480,) 69


# Table 6, CTR, NDCG@1, NDCG@4 Columns

In [47]:
for name, df in scores:
    actual = df[['user', 'arm', 'reward']].drop_duplicates()
    predicted = df[['user', 'item_id', 'probability']]
    predicted.columns = actual.columns
    ctr = BinaryRecoMetrics.CTR(user_id_column='user', item_id_column='arm', click_column="reward", k=1).get_score(actual, predicted)
    ndcg = RankingRecoMetrics.NDCG(user_id_column='user', item_id_column='arm', click_column="reward", k=4).get_score(actual, predicted)
    precision = RankingRecoMetrics.Precision(user_id_column='user', item_id_column='arm', click_column="reward", k=1).get_score(actual, predicted)
    recall = RankingRecoMetrics.Recall(user_id_column='user', item_id_column='arm', click_column="reward", k=1).get_score(actual, predicted)
    ndcg1 = RankingRecoMetrics.NDCG(user_id_column='user', item_id_column='arm', click_column="reward", k=1).get_score(actual, predicted)
    ctr4 = BinaryRecoMetrics.CTR(user_id_column='user', item_id_column='arm', click_column="reward", k=4).get_score(actual, predicted)

    print(name, ':', ctr, ndcg, precision, recall, ndcg1, ctr4)

Mac : 0.42105263157894735 0.6295164130273118 0.13793103448275862 0.13793103448275862 0.13793103448275862 0.48333333333333334
Windows : 0.3888888888888889 0.6197004916147841 0.1206896551724138 0.1206896551724138 0.1206896551724138 0.48333333333333334
SageMaker : 0.42105263157894735 0.6295164130273118 0.13793103448275862 0.13793103448275862 0.13793103448275862 0.48333333333333334
Red Hat : 0.42105263157894735 0.6295164130273118 0.13793103448275862 0.13793103448275862 0.13793103448275862 0.48333333333333334
Darwin : 0.47619047619047616 0.6320181033142886 0.1724137931034483 0.1724137931034483 0.1724137931034483 0.48333333333333334


In [48]:
temp = mac_df[['user', 'arm', 'reward']].drop_duplicates()
temp['reward'].mean()

0.48333333333333334

# Table 5, Prediction Comparisons

In [45]:
results3 = []
env = ['Mac', 'Windows', 'SageMaker', 'Red Hat', 'Darwin']

for i, name_i in enumerate(env):
    a = name_i + 'pred'
    for j, name_j in enumerate(env):
        if j > i:
            b = name_j + 'pred'
            predictions = df_full[a] == df_full[b]
            results3.append((name_i, name_j, predictions))
            
for a, b, c in results3:
    print(a, b, c.shape, str(c.sum()))

Mac Windows (480,) 476
Mac SageMaker (480,) 480
Mac Red Hat (480,) 480
Mac Darwin (480,) 441
Windows SageMaker (480,) 476
Windows Red Hat (480,) 476
Windows Darwin (480,) 437
SageMaker Red Hat (480,) 480
SageMaker Darwin (480,) 441
Red Hat Darwin (480,) 441


# Cholesky

In [2]:
mac = pickle.load(open(os.path.join('output', 'mac_ch_expectations.pkl'), 'rb'))
win = pickle.load(open(os.path.join('output', 'win_ch_expectations.pkl'), 'rb'))
sgm = pickle.load(open(os.path.join('output', 'sgm_ch_expectations.pkl'), 'rb'))
rh = pickle.load(open(os.path.join('output', 'rh_ch_expectations.pkl'), 'rb'))
dar = pickle.load(open(os.path.join('output', 'dar_ch_expectations.pkl'), 'rb'))

In [3]:
responses = pd.read_csv('simulated_data.csv')
test = responses[responses['set']=='test']
test.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,arm,reward,set
280,-0.437539,1.579266,1.855246,6.049628,0.178026,-1.985470,0.005097,0.571541,1.184153,0.250244,...,-0.815639,-2.154333,-0.302746,-2.941561,-1.879810,1.794686,1.415656,0,1,test
281,-1.483903,2.533863,1.334310,-2.786843,-3.153749,-10.011978,-1.486536,2.942048,1.658577,0.282615,...,-7.705325,-2.362107,-0.077421,1.519168,-0.288540,0.164909,-3.257780,2,0,test
282,-1.393583,5.561957,-0.440106,-0.586712,3.047809,-1.553686,1.784540,-0.724047,3.457133,-0.520384,...,-0.580570,0.049443,0.357533,-1.653148,-0.676257,3.890172,-1.096128,3,0,test
283,-4.979037,1.186080,-0.970855,-1.207987,-0.060639,0.088849,-1.570659,1.031042,-0.470511,-1.387079,...,0.173680,-0.856460,-1.225014,0.115370,0.317427,0.476862,3.814958,1,0,test
284,-0.785594,0.057483,-1.800295,-4.470890,-0.046001,-3.470062,-3.025259,0.005172,-2.017571,-1.083593,...,4.259867,2.013612,-0.559288,-3.612187,-0.884423,-0.787468,-1.185029,2,0,test


In [4]:
test = test[['arm', 'reward']]

In [5]:
test['user'] = [i for i in range(test.shape[0])]

In [6]:
test.reset_index(inplace=True, drop=True)

In [7]:
mac_df = pd.DataFrame(mac)
win_df = pd.DataFrame(win)
sgm_df = pd.DataFrame(sgm)
rh_df = pd.DataFrame(rh)
dar_df = pd.DataFrame(dar)

mac_df = mac_df.merge(test, how='left', left_index=True, right_index=True)
win_df = win_df.merge(test, how='left', left_index=True, right_index=True)
sgm_df = sgm_df.merge(test, how='left', left_index=True, right_index=True)
rh_df = rh_df.merge(test, how='left', left_index=True, right_index=True)
dar_df = dar_df.merge(test, how='left', left_index=True, right_index=True)

mac_df.head()

,0,1,2,3,arm,reward,user
0,-9.072648,1.178444,1.364816,1.768326,0,1,0
1,-7.657203,3.904120,-1.579630,-5.410818,2,0,1
2,8.462115,7.213985,0.499564,-4.988740,3,0,2
3,13.594220,-1.645057,-0.060344,12.880111,1,0,3
4,6.430210,-0.850406,-0.614397,0.122754,2,0,4


In [8]:
mac_df = mac_df.melt(id_vars=['user', 'arm', 'reward'], var_name='item_id', value_name='raw_score')
win_df = win_df.melt(id_vars=['user', 'arm', 'reward'], var_name='item_id', value_name='raw_score')
sgm_df = sgm_df.melt(id_vars=['user', 'arm', 'reward'], var_name='item_id', value_name='raw_score')
rh_df = rh_df.melt(id_vars=['user', 'arm', 'reward'], var_name='item_id', value_name='raw_score')
dar_df = dar_df.melt(id_vars=['user', 'arm', 'reward'], var_name='item_id', value_name='raw_score')

In [9]:
mac_df['probability'] = mac_df.apply(lambda x: 1/(1 + np.exp(-x['raw_score'])), axis=1)

In [10]:
win_df['probability'] = win_df.apply(lambda x: 1/(1 + np.exp(-x['raw_score'])), axis=1)
sgm_df['probability'] = sgm_df.apply(lambda x: 1/(1 + np.exp(-x['raw_score'])), axis=1)
rh_df['probability'] = rh_df.apply(lambda x: 1/(1 + np.exp(-x['raw_score'])), axis=1)
dar_df['probability'] = dar_df.apply(lambda x: 1/(1 + np.exp(-x['raw_score'])), axis=1)

In [11]:
mac_df['pred'] = mac_df['probability'] >= 0.5
win_df['pred'] = win_df['probability'] >= 0.5
sgm_df['pred'] = sgm_df['probability'] >= 0.5
rh_df['pred'] = rh_df['probability'] >= 0.5
dar_df['pred'] = dar_df['probability'] >= 0.5

# Table 6, Clicks Column

In [12]:
mac_df['pred'].sum()

230

In [13]:
win_df['pred'].sum()

230

In [14]:
sgm_df['pred'].sum()

230

In [15]:
rh_df['pred'].sum()

230

In [16]:
dar_df['pred'].sum()

230

In [18]:
scores = [('Mac', mac_df), ('Windows', win_df), ('SageMaker', sgm_df), ('Red Hat', rh_df), ('Darwin', dar_df)]

df_full = None

for name, df in scores:
    df_temp = df[['user', 'item_id', 'raw_score', 'probability', 'pred']]
    df_temp.columns = [name + c if c not in ['user', 'item_id'] else c for c in df_temp.columns]
    
    if df_full is None:
        df_full = df_temp
    else:
        df_full = df_full.merge(df_temp, how='left', on=['user', 'item_id'])

In [19]:
df_full.head()

,user,item_id,Macraw_score,Macprobability,Macpred,Windowsraw_score,Windowsprobability,Windowspred,SageMakerraw_score,SageMakerprobability,SageMakerpred,Red Hatraw_score,Red Hatprobability,Red Hatpred,Darwinraw_score,Darwinprobability,Darwinpred
0,0,0,-9.072648,0.000115,False,-9.072648,0.000115,False,-9.072648,0.000115,False,-9.072648,0.000115,False,-9.072648,0.000115,False
1,1,0,-7.657203,0.000472,False,-7.657203,0.000472,False,-7.657203,0.000472,False,-7.657203,0.000472,False,-7.657203,0.000472,False
2,2,0,8.462115,0.999789,True,8.462115,0.999789,True,8.462115,0.999789,True,8.462115,0.999789,True,8.462115,0.999789,True
3,3,0,13.594220,0.999999,True,13.594220,0.999999,True,13.594220,0.999999,True,13.594220,0.999999,True,13.594220,0.999999,True
4,4,0,6.430210,0.998390,True,6.430210,0.998390,True,6.430210,0.998390,True,6.430210,0.998390,True,6.430210,0.998390,True


In [20]:
results = []
env = ['Mac', 'Windows', 'SageMaker', 'Red Hat', 'Darwin']

for i, name_i in enumerate(env):
    a = name_i + 'raw_score'
    for j, name_j in enumerate(env):
        if j > i:
            b = name_j + 'raw_score'
            similarity = np.isclose(df_full[a], df_full[b])
            results.append((name_i, name_j, similarity))
            
            
for a, b, c in results:
    print(a, b, c.shape, str(c.sum()))

Mac Windows (480,) 480
Mac SageMaker (480,) 480
Mac Red Hat (480,) 480
Mac Darwin (480,) 480
Windows SageMaker (480,) 480
Windows Red Hat (480,) 480
Windows Darwin (480,) 480
SageMaker Red Hat (480,) 480
SageMaker Darwin (480,) 480
Red Hat Darwin (480,) 480


# Table 6, Cholesky Row, CTR, NDCG@1, NDCG@4 Columns

In [21]:
for name, df in scores:
    actual = df[['user', 'arm', 'reward']].drop_duplicates()
    predicted = df[['user', 'item_id', 'probability']]
    predicted.columns = actual.columns
    ctr = BinaryRecoMetrics.CTR(user_id_column='user', item_id_column='arm', click_column="reward", k=1).get_score(actual, predicted)
    ndcg = RankingRecoMetrics.NDCG(user_id_column='user', item_id_column='arm', click_column="reward", k=4).get_score(actual, predicted)
    precision = RankingRecoMetrics.Precision(user_id_column='user', item_id_column='arm', click_column="reward", k=1).get_score(actual, predicted)
    recall = RankingRecoMetrics.Recall(user_id_column='user', item_id_column='arm', click_column="reward", k=1).get_score(actual, predicted)
    ndcg1 = RankingRecoMetrics.NDCG(user_id_column='user', item_id_column='arm', click_column="reward", k=1).get_score(actual, predicted)
    ctr4 = BinaryRecoMetrics.CTR(user_id_column='user', item_id_column='arm', click_column="reward", k=4).get_score(actual, predicted)

    print(name, ':', ctr, ndcg, precision, recall, ndcg1, ctr4)

Mac : 0.3888888888888889 0.6321075028884455 0.2413793103448276 0.2413793103448276 0.2413793103448276 0.48333333333333334
Windows : 0.3888888888888889 0.6321075028884455 0.2413793103448276 0.2413793103448276 0.2413793103448276 0.48333333333333334
SageMaker : 0.3888888888888889 0.6321075028884455 0.2413793103448276 0.2413793103448276 0.2413793103448276 0.48333333333333334
Red Hat : 0.3888888888888889 0.6321075028884455 0.2413793103448276 0.2413793103448276 0.2413793103448276 0.48333333333333334
Darwin : 0.3888888888888889 0.6321075028884455 0.2413793103448276 0.2413793103448276 0.2413793103448276 0.48333333333333334


# Table 6, Cholesky Row, Accuracy, Precision, Recall, and AUC Columns

In [22]:
scores = [('Mac', mac_df), ('Windows', win_df), ('SageMaker', sgm_df), ('Red Hat', rh_df), ('Darwin', dar_df)]
for name, df in scores:

    temp_idx = df['arm'] == df['item_id']
    temp = df[temp_idx]
    acc = BinaryClassificationMetrics.Accuracy().get_score(temp['pred'].tolist(), temp['reward'].tolist())
    f1 = BinaryClassificationMetrics.F1().get_score(temp['pred'].tolist(), temp['reward'].tolist())
    precision = BinaryClassificationMetrics.Precision().get_score(temp['pred'].tolist(), temp['reward'].tolist())
    recall = BinaryClassificationMetrics.Recall().get_score(temp['pred'].tolist(), temp['reward'].tolist())
    auc = BinaryClassificationMetrics.AUC().get_score(temp['pred'].tolist(), temp['reward'].tolist())
    print(name, ':', acc, f1, precision, recall, auc)

Mac : 0.425 0.448 0.4827586206896552 0.417910447761194 0.4259363559560687
Windows : 0.425 0.448 0.4827586206896552 0.417910447761194 0.4259363559560687
SageMaker : 0.425 0.448 0.4827586206896552 0.417910447761194 0.4259363559560687
Red Hat : 0.425 0.448 0.4827586206896552 0.417910447761194 0.4259363559560687
Darwin : 0.425 0.448 0.4827586206896552 0.417910447761194 0.4259363559560687
